In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
!pip install transformers==4.28.1 datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from transformers import BertTokenizer, BertModel, AdamW, BertConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
from sklearn.utils import resample
import copy
import spacy
from collections import Counter

from transformers.utils import logging
logging.set_verbosity_error()
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
model_name = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [6]:
import pandas as pd
import numpy as np

df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/wac_project/QA_v2/df_sentences_context.xlsx')
df["text"] = df["text"].fillna("").astype(str)
df["context"] = df["context"].fillna("").astype(str)

label_list = list(df['label'].unique())

label2id = {'Evidence': 0,
            'Organizational Framework': 1,
            'Reasoning': 2,
            'Rhetorical Structure - Focus': 3,
            'Rhetorical Structure - Progression': 4,
            'Thesis': 5,
            'normal_text': 6,
            'grouped': 7}

id2label = {i: label for label, i in label2id.items()}

evidence = 'Evidence'
org_framework ='Organizational Framework'
reasoning = 'Reasoning'
focus = 'Rhetorical Structure - Focus'
progression = 'Rhetorical Structure - Progression'
thesis = 'Thesis'
normal_text = 'normal_text'
grouped = 'grouped'

In [7]:
def convert_df_to_binary_on_column(df, col, val):
  df_temp = df.copy(deep=True)
  df_temp[col] = df_temp[col].apply(lambda x: 1 if x != val else 0)
  return df_temp

In [8]:
def split_data(df, train_size=0.7, test_size=0.15, eval_size=0.15, strategy=None, random_state=42):
  if strategy == 'id':
    train_id, temp_id = train_test_split(
        df['file_id'].unique(),
        train_size=train_size,
        random_state=random_state
        )

    val_id, test_id = train_test_split(
        temp_id,
        test_size=np.round(test_size / (1 - train_size), 1),
        random_state=random_state
    )

    train_df = df[df['file_id'].isin(train_id)].copy(deep=True)
    val_df = df[df['file_id'].isin(val_id)].copy(deep=True)
    test_df = df[df['file_id'].isin(test_id)].copy(deep=True)

    return train_df, test_df, val_df

  else:
    train_df, temp_df = train_test_split(
        df,
        test_size=1-train_size,
        random_state=random_state
        )
    val_df, test_df = train_test_split(
        temp_df,
        test_size=np.round(test_size / (1 - train_size), 1),
        random_state=random_state
    )
    return train_df.copy(deep=True), test_df.copy(deep=True), val_df.copy(deep=True)

In [9]:
def balance_df(df):
  assert set(df['y'].unique()).issubset({0, 1})
  target_ratio = 0.15
  ones_df = df[df['y'] == 1]
  zeros_df = df[df['y'] == 0]
  n_ones = len(ones_df)
  total_desired = int(n_ones / target_ratio)
  n_zeros_needed = total_desired - n_ones
  sampled_zeros_df = zeros_df.sample(n=n_zeros_needed, random_state=42) # Downsample 0s
  balanced_df = pd.concat([ones_df, sampled_zeros_df]).sample(frac=1, random_state=42).reset_index(drop=True) # Combine
  return balanced_df

def perfect_balancing_downsampling(df, y='label', label=None, random_state=42):
  df_value_counts = df[y].value_counts()
  if label is not None:
     min_count = df_value_counts.loc[label]
  else:
    min_count = min(df_value_counts)

  labels = df[y].unique()
  df_list = []
  for label in labels:
    df_label = df[df[y] == label]
    # n = min_count if min_count <= df_value_counts[label] else df_value_counts[label]
    df_label = df_label.sample(n=min_count, replace=True, random_state=random_state)
    df_list.append(df_label)
  df_final = pd.concat(df_list).reset_index(drop=True).copy(deep=True)
  return df_final

def add_pos_tags(text):
    doc = nlp(text)
    return " ".join([f"{token.text}/{token.pos_}" for token in doc])

def replace_with_pos_tags(text):
    doc = nlp(text)
    return " ".join([f"{token.pos_}" for token in doc])

In [10]:
def encode_df(df_subset, encode_pair=False):
    df_subset = df_subset.copy()
    df_subset["text"] = df_subset["text"].fillna("").astype(str)
    df_subset["context"] = df_subset["context"].fillna("").astype(str)

    if encode_pair:
        ret = tokenizer(
            list(df_subset["text"]),
            list(df_subset["context"]),
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt",
            return_overflowing_tokens=False)
    else:
        ret = tokenizer(
            list(df_subset["text"]),
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt",
            return_overflowing_tokens=False)

    return ret

In [11]:
add_pos = False
encode_pair = False

In [12]:
if add_pos:
  print('Add POS Tags - ', add_pos)
  df['text'] = df['text'].apply(replace_with_pos_tags)
  df['context'] = df['context'].apply(add_pos_tags)

train_df, test_df, val_df = split_data(df, train_size=0.7, test_size=0.15, eval_size=0.15, strategy='id', random_state=42)

# train_df = perfect_balancing_downsampling(train_df)

# train_df['label'] = train_df['label'].map(label2id)
# test_df['label'] = test_df['label'].map(label2id)
# val_df['label'] = val_df['label'].map(label2id)

# print('Sentence Pair Encoding - ', encode_pair)

# train_encodings = encode_df(train_df, encode_pair)
# val_encodings = encode_df(val_df, encode_pair)
# test_encodings = encode_df(test_df, encode_pair)

# train_labels = torch.tensor(train_df["label"].astype(int).values).to(device)
# val_labels = torch.tensor(val_df["label"].astype(int).values).to(device)
# test_labels = torch.tensor(test_df["label"].astype(int).values).to(device)

# train_inputs = {k: v.to(device) for k, v in train_encodings.items()}
# val_inputs = {k: v.to(device) for k, v in val_encodings.items()}
# test_inputs = {k: v.to(device) for k, v in test_encodings.items()}

In [13]:
class BertMultiClassClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_output = outputs.pooler_output
        x = self.dropout(cls_output)
        return self.classifier(x)

In [14]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [15]:
from sklearn.utils.class_weight import compute_class_weight

num_labels = [0, 1, 2, 3, 4, 5, 6] # train_df['label'].unique()
num_labels.sort()

class_weights = compute_class_weight(class_weight='balanced', classes=np.array(label_list), y=train_df['label'])

# **Basic Methods**

In [16]:
# num_labels = list(label2id.values())
# string_labels = [id2label[i] for i in num_labels]
# num_classes = len(label2id)

def train_model(model, train_inputs, train_labels, val_inputs, val_labels, batch_size=32, weighted_loss=False, criterion=None):
  optimizer = AdamW(model.parameters(), lr=2e-5)

  if criterion is None:
    if weighted_loss:
      criterion = nn.CrossEntropyLoss(weight=torch.from_numpy(class_weights).float().to(device))
    else:
      criterion = nn.CrossEntropyLoss()

  epochs = 50
  patience = 3
  best_val_loss = float("inf")
  patience_counter = 0
  best_model_state = None

  model.train()

  for epoch in range(epochs):
      total_train_loss = 0
      model.train()

      for i in range(0, len(train_labels), batch_size):
          input_batch = {k: v[i:i+batch_size] for k, v in train_inputs.items()}
          label_batch = train_labels[i:i+batch_size]

          optimizer.zero_grad()
          logits = model(**input_batch)
          loss = criterion(logits, label_batch)
          loss.backward()
          optimizer.step()

          total_train_loss += loss.item()

      avg_train_loss = total_train_loss / (len(train_labels) // batch_size + 1)

      torch.cuda.empty_cache()
      torch.cuda.ipc_collect()

      model.eval()
      val_loss = 0

      with torch.no_grad():
          for i in range(0, len(val_labels), batch_size):
              batch = {k: v[i:i+batch_size] for k, v in val_inputs.items()}
              label_batch = val_labels[i:i+batch_size]

              logits = model(**batch)
              loss = criterion(logits, label_batch)

              val_loss += loss.item()

      avg_val_loss = val_loss / (len(val_labels) // batch_size + 1)
      print(f"Epoch {epoch+1}/{epochs} — Train Loss: {avg_train_loss:.4f} — Val Loss: {avg_val_loss:.4f}")


      if val_loss < best_val_loss:
          best_val_loss = val_loss
          patience_counter = 0
          best_model_state = copy.deepcopy(model.state_dict())
      else:
          patience_counter += 1
          if patience_counter >= patience:
              print("Early stopping triggered.")
              break

      torch.cuda.empty_cache()
      torch.cuda.ipc_collect()

  if best_model_state is not None:
      model.load_state_dict(best_model_state)

  return model, best_val_loss

def test_model(model, test_inputs, test_labels, batch_size=32):
  model.eval()
  all_preds = []
  all_true = []

  with torch.no_grad():
      for i in range(0, len(test_labels), batch_size):
          batch_inputs = {k: v[i:i+batch_size] for k, v in test_inputs.items()}
          label_batch = test_labels[i:i+batch_size]

          logits = model(**batch_inputs)#.logits
          preds = torch.argmax(logits, dim=1)

          all_preds.extend(preds.cpu().numpy())
          all_true.extend(label_batch.cpu().numpy())

  torch.cuda.empty_cache()
  torch.cuda.ipc_collect()

  y_true = np.array(all_true)
  y_pred = np.array(all_preds)

  performance_metrics = np.asarray(precision_recall_fscore_support(y_true, y_pred, zero_division=0)) # , labels=num_labels

  return performance_metrics.T, y_pred

# **Ensemble Methods**

In [17]:
def train_ensemble_model(models, train_inputs_list, train_labels_list, val_input, val_label, weighted_loss=False):
  trained_models = []
  for model, train_inputs, train_labels in zip(models, train_inputs_list, train_labels_list):
    trained_model, best_val_loss = train_model(model, train_inputs, train_labels, val_input, val_label, weighted_loss=weighted_loss)
    trained_models.append(trained_model)
    print()

  return trained_models

def test_ensemble_model(models, test_inputs, test_labels, batch_size=32, strategy='hard vote'):
    def hard_voting(tensors):
        np_tensors = [t.numpy() for t in tensors]
        stacked = np.stack(np_tensors)
        result = []
        for i in range(stacked.shape[1]):
            col = stacked[:, i]
            count = Counter(col)
            max_freq = max(count.values())
            most_common_values = [val for val, freq in count.items() if freq == max_freq]
            result.append(min(most_common_values))
        return torch.tensor(result)

    def soft_voting(probs_list):
        stacked = torch.stack(probs_list)
        avg_probs = stacked.mean(dim=0)
        return torch.argmax(avg_probs, dim=1)

    y_true = test_labels.cpu().numpy()
    model_predictions = []

    for model in models:
        model.eval()
        all_preds = []

        with torch.no_grad():
            for i in range(0, len(test_labels), batch_size):
                batch_inputs = {k: v[i:i+batch_size] for k, v in test_inputs.items()}
                logits = model(**batch_inputs)

                if strategy == 'hard vote':
                    preds = torch.argmax(logits, dim=1)
                    all_preds.extend(preds.cpu().numpy())
                elif strategy == 'soft vote':
                    probs = F.softmax(logits, dim=1)
                    all_preds.append(probs.cpu())
                else:
                    raise ValueError(f"Unknown strategy: {strategy}")

        if strategy == 'hard vote':
            model_predictions.append(torch.tensor(all_preds))
        elif strategy == 'soft vote':
            model_predictions.append(torch.cat(all_preds, dim=0))

        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

    if strategy == 'hard vote':
        final_predictions = hard_voting(model_predictions).numpy()
    else:
        final_predictions = soft_voting(model_predictions).cpu().numpy()

    performance_metrics = np.asarray(
        precision_recall_fscore_support(y_true, final_predictions, labels=num_labels, zero_division=0)
    )

    return performance_metrics.T, final_predictions, y_true

def test_hierarchical_classification_ensemble(models, test_inputs, test_labels, c1_reverse_label_map, c2_reverse_label_map, c3_reverse_label_map):
  batch_size = 1
  classifier_1, classifier_2, classifier_3 = models
  predictions = []
  for i in range(0, len(test_labels), batch_size):
      batch_inputs = {k: v[i:i+batch_size] for k, v in test_inputs.items()}
      label_batch = test_labels[i:i+batch_size]

      logits = classifier_1(**batch_inputs)
      c1_pred = int(torch.argmax(logits))
      if c1_pred == 0:
          predictions.append(c1_reverse_label_map[c1_pred])
      else:
          logits = classifier_2(**batch_inputs)
          c2_pred = int(torch.argmax(logits))
          if c2_pred != 2:
            predictions.append(c2_reverse_label_map[c2_pred])
          else:
            logits = classifier_3(**batch_inputs)
            c3_pred = int(torch.argmax(logits))
            predictions.append(c3_reverse_label_map[c3_pred])

  y_true = test_labels.cpu().numpy()
  y_pred = np.array(predictions)

  performance_metrics = np.asarray(precision_recall_fscore_support(y_true, y_pred, zero_division=0))

  return performance_metrics.T, y_pred

In [18]:
# model = BertMultiClassClassifier(num_classes).to(device)
# test_ensemble_model([model]*3, test_inputs, test_labels, batch_size=32)

In [19]:
def create_c1_df(df, y='label'):
  new_df = df.copy(deep=True)
  new_df[y] = new_df[y].apply(lambda x: normal_text if x == normal_text else grouped)
  return new_df

def create_c2_df(df, y='label'):
  new_df = df.copy(deep=True)
  new_df = new_df[new_df[y] != normal_text ]
  new_df[y] = new_df[y].apply(lambda x: x if x in (reasoning, evidence) else grouped)
  return new_df

def create_c3_df(df, y='label'):
  new_df = df.copy(deep=True)
  new_df = new_df[new_df[y].isin((focus, thesis, progression, org_framework))]
  return new_df

def create_voting_system_dfs(df, y='label'):
  label_counts = df[y].value_counts().to_dict()
  min_class_size = min(label_counts.values())
  all_labels = label_counts.keys()

  df1 = pd.concat([ resample(df[df[y] == label], n_samples=150, random_state=42) for label in all_labels ]).copy(deep=True)

  mild_targets = {
      normal_text : 1000,
      reasoning: 1368,
      evidence: 1227,
      focus: 500,
      thesis: 300,
      progression: 300,
      org_framework: 300
  }

  df2 = pd.concat([
      resample(
          df[df[y] == label],
          n_samples=count,
          random_state=43,
          replace=True
      )
      for label, count in mild_targets.items()
      # if label in df[y].unique() and len(df[df[y] == label]) > 0
  ]).copy(deep=True)

  df3 = [df[df[y] == 'normal_text']]
  for label in all_labels:
      if label != 'normal_text':
          original = df[df[y] == label]
          n = max(1, int(0.7 * len(original)))  # avoid 0
          df3.append(resample(original, n_samples=n, random_state=44))
  df3 = pd.concat(df3).copy(deep=True)

  df_overminor = []
  if normal_text in df[y].unique():
      norm_count = min(2000, len(df[df[y] == normal_text]))
      df_overminor.append(resample(df[df[y] == normal_text], n_samples=norm_count, random_state=45))

  for label in [reasoning, evidence, focus]:
      if label in df[y].unique():
          df_overminor.append(df[df[y] == label])

  for label in [thesis, progression, org_framework]:
      if label in df[y].unique():
          class_df = df[df[y] == label]
          n = 3 * len(class_df)
          if n > 0:
              df_overminor.append(resample(class_df, replace=True, n_samples=n, random_state=46))

  df4 = pd.concat(df_overminor).copy(deep=True)

  df5 = resample(df, n_samples=len(df), replace=True, stratify=df[y], random_state=49).copy(deep=True)

  return df1, df2, df3, df4, df5

def create_c_encodings(df, create_c_df, c_label_map, y='label'):
  new_df = create_c_df(df)
  new_df.loc[:, y] = new_df[y].map(label2id)
  new_df.loc[:, y] = new_df[y].map(c_label_map)
  encodings = encode_df(new_df)
  labels = torch.tensor(new_df[y].astype(int).values).to(device)
  inputs = {k: v.to(device) for k, v in encodings.items()}
  return inputs, labels

In [20]:
# train_df, test_df, val_df = split_data(df, train_size=0.7, test_size=0.15, eval_size=0.15, strategy='id', random_state=42)

# c1_label_map = {6:0, 7:1}
# c1_reverse_label_map = {i: label for label, i in c1_label_map.items()}

# c1_train_inputs, c1_train_labels = create_c_encodings(train_df, create_c1_df, c1_label_map)
# c1_val_inputs, c1_val_labels = create_c_encodings(val_df, create_c1_df, c1_label_map)
# c1_test_inputs, c1_test_labels = create_c_encodings(test_df, create_c1_df, c1_label_map)

# c1_num_classes = len(c1_label_map)
# cl_classifier = BertMultiClassClassifier(len(c1_num_classes)).to(device)
# cl_classifier_trained, c1_best_val_loss = train_model(cl_classifier, c1_train_inputs, c1_train_labels, c1_val_inputs, c1_val_labels, batch_size=32, weighted_loss=False)

# performnce_matrix, y_preds = test_model(cl_classifier_trained, c1_test_inputs, c1_test_labels, batch_size=32)
# print(pd.DataFrame(performnce_matrix, columns=['precision', 'recall', 'F1', 'support']).rename(index=c1_reverse_label_map).rename(index=id2label))

In [21]:
# y = 'label'
# batch_size = 32
# n_run_metrics = []
# for run in range(3):
#   print(f'Run {run+1}')

#   # Classifier 1
#   c1_label_map = {6:0, 7:1}
#   c1_reverse_label_map = {i: label for label, i in c1_label_map.items()}

#   c1_train_inputs, c1_train_labels = create_c_encodings(train_df, create_c1_df, c1_label_map)
#   c1_val_inputs, c1_val_labels = create_c_encodings(val_df, create_c1_df, c1_label_map)

#   c1_num_classes = len(c1_label_map)
#   cl_classifier = BertMultiClassClassifier(c1_num_classes).to(device)
#   cl_classifier_trained, c1_best_val_loss = train_model(cl_classifier, c1_train_inputs, c1_train_labels, c1_val_inputs, c1_val_labels, batch_size=batch_size, weighted_loss=False)

#   print()
#   # Classifier 2
#   c2_label_map = {2:0, 0:1, 7:2}
#   c2_reverse_label_map = {i: label for label, i in c2_label_map.items()}

#   c2_train_inputs, c2_train_labels = create_c_encodings(train_df, create_c2_df, c2_label_map)
#   c2_val_inputs, c2_val_labels = create_c_encodings(val_df, create_c2_df, c2_label_map)

#   c2_num_classes = len(c2_label_map)
#   c2_classifier = BertMultiClassClassifier(c2_num_classes).to(device)
#   c2_classifier_trained, c2_best_val_loss = train_model(c2_classifier, c2_train_inputs, c2_train_labels, c2_val_inputs, c2_val_labels, batch_size=batch_size, weighted_loss=False)

#   print()
#   # Classifier 3
#   c3_label_map = {3:0, 5:1, 4:2, 1:3}
#   c3_reverse_label_map = {i: label for label, i in c3_label_map.items()}

#   c3_train_inputs, c3_train_labels = create_c_encodings(train_df, create_c3_df, c3_label_map)
#   c3_val_inputs, c3_val_labels = create_c_encodings(val_df, create_c3_df, c3_label_map)

#   c3_num_classes = len(c3_label_map)
#   c3_classifier = BertMultiClassClassifier(c3_num_classes).to(device)
#   c3_classifier_trained, c3_best_val_loss = train_model(c3_classifier, c3_train_inputs, c3_train_labels, c3_val_inputs, c3_val_labels, batch_size=batch_size, weighted_loss=False)

#   trained_classifier = [cl_classifier_trained, c2_classifier_trained, c3_classifier_trained]
#   current_run_metrics, test_predictions = test_hierarchical_classification_ensemble(trained_classifier, test_inputs, test_labels, c1_reverse_label_map, c2_reverse_label_map, c3_reverse_label_map)
#   n_run_metrics.append(current_run_metrics)
#   print(pd.DataFrame(current_run_metrics, columns=['precision', 'recall', 'F1', 'support']).rename(index=id2label))

# avg_n_run_metrics = pd.DataFrame(np.mean(np.asarray(n_run_metrics), axis=0), columns=['precision','recall','F1','support']) #  index=label_mapping
# print('\nAverge Metrics')
# print(avg_n_run_metrics.rename(index=id2label))

In [22]:
# batch_size = 32
# n_run_metrics = []
# for run in range(3):
#   print(f'Run {run+1}')

#   custom_train_df = train_df.copy(deep=True)
#   custom_val_df = val_df.copy(deep=True)
#   custom_test_df = test_df.copy(deep=True)

#   label_remap = {1:0, 3:1, 4:2, 5:3}
#   reverse_label_remap = {i: label for label, i in label_remap.items()}
#   label_mapping = { id2label[j]:i for i, j in reverse_label_remap.items()}
#   print("Label remap:", label_remap)

#   custom_train_df = custom_train_df[custom_train_df['label'].isin([focus, thesis, progression, org_framework])]
#   # custom_train_df.loc[:, 'label'] = custom_train_df['label'].apply(lambda x: x if x in (reasoning, evidence) else grouped)
#   custom_train_df.loc[:, 'label'] = custom_train_df['label'].map(label2id)
#   custom_train_df.loc[:, 'label'] = custom_train_df['label'].map(label_remap)
#   train_encodings = encode_df(custom_train_df, encode_pair)
#   train_labels = torch.tensor(custom_train_df["label"].astype(int).values).to(device)
#   train_inputs = {k: v.to(device) for k, v in train_encodings.items()}

#   custom_val_df = custom_val_df[custom_val_df['label'].isin([focus, thesis, progression, org_framework])]
#   # custom_val_df.loc[:, 'label'] = custom_val_df['label'].apply(lambda x: x if x in (reasoning, evidence) else grouped)
#   custom_val_df.loc[:, 'label'] = custom_val_df['label'].map(label2id)
#   custom_val_df.loc[:, 'label'] = custom_val_df['label'].map(label_remap)
#   val_encodings = encode_df(custom_val_df, encode_pair)
#   val_labels = torch.tensor(custom_val_df["label"].astype(int).values).to(device)
#   val_inputs = {k: v.to(device) for k, v in val_encodings.items()}

#   custom_test_df = custom_test_df[custom_test_df['label'].isin([focus, thesis, progression, org_framework])]
#   # custom_test_df.loc[:, 'label'] = custom_test_df['label'].apply(lambda x: x if x in (reasoning, evidence) else grouped)
#   custom_test_df.loc[:, 'label'] = custom_test_df['label'].map(label2id)
#   custom_test_df.loc[:, 'label'] = custom_test_df['label'].map(label_remap)
#   test_encodings = encode_df(custom_test_df, encode_pair)
#   test_labels = torch.tensor(custom_test_df["label"].astype(int).values).to(device)
#   test_inputs = {k: v.to(device) for k, v in test_encodings.items()}

#   num_classes = len(label_remap)
#   model = BertMultiClassClassifier(num_classes).to(device)

#   trained_model, best_val_loss = train_model(model, train_inputs, train_labels, val_inputs, val_labels, batch_size=batch_size, weighted_loss=False)

#   current_run_metrics, _ = test_model(trained_model, test_inputs, test_labels, batch_size=batch_size)
#   n_run_metrics.append(current_run_metrics)
#   print('Test Results:')
#   print(pd.DataFrame(current_run_metrics, columns=['precision', 'recall', 'F1', 'support'], index=label_mapping))

# avg_n_run_metrics = pd.DataFrame(np.mean(np.asarray(n_run_metrics), axis=0), columns=['precision','recall','F1','support'], index=label_mapping)
# print('\nAverge Metrics')
# print(avg_n_run_metrics)

In [23]:
n_run_metrics = []

train_df, test_df, val_df = split_data(df, train_size=0.7, test_size=0.15, eval_size=0.15, strategy='id', random_state=42)

test_df['label'] = test_df['label'].map(label2id)
val_df['label'] = val_df['label'].map(label2id)

val_encodings = encode_df(val_df, encode_pair)
test_encodings = encode_df(test_df, encode_pair)

val_labels = torch.tensor(val_df["label"].astype(int).values).to(device)
test_labels = torch.tensor(test_df["label"].astype(int).values).to(device)

val_inputs = {k: v.to(device) for k, v in val_encodings.items()}
test_inputs = {k: v.to(device) for k, v in test_encodings.items()}

dfs = create_voting_system_dfs(train_df, y='label')

for run in range(1):
  print(f'Run {run+1}')

  num_classes = len(num_labels)

  train_inputs_list, train_labels_list = [], []
  val_inputs_list, val_labels_list = [], []
  models = []
  for df in dfs:
    df['label'] = df['label'].map(label2id)
    aug_encodings = encode_df(df, encode_pair)

    train_labels = torch.tensor(df["label"].astype(int).values).to(device)
    train_inputs = {k: v.to(device) for k, v in aug_encodings.items()}

    train_inputs_list.append(train_inputs)
    train_labels_list.append(train_labels)
    models.append(BertMultiClassClassifier(num_classes).to(device))

  model = BertMultiClassClassifier(num_classes).to(device)
  trained_models = train_ensemble_model(models, train_inputs_list, train_labels_list, val_inputs, val_labels)

  print('Ensemble Testing')
  current_run_metrics, _, _ = test_ensemble_model(trained_models, test_inputs, test_labels, batch_size=32, strategy='hard vote')

  print('Hard Vote Evaluation Results:')
  print(pd.DataFrame(current_run_metrics, columns=['precision', 'recall', 'F1', 'support']).rename(index=id2label))

  current_run_metrics, _, _ = test_ensemble_model(trained_models, test_inputs, test_labels, batch_size=32, strategy='soft vote')

  print('Soft Vote Evaluation Results:')
  print(pd.DataFrame(current_run_metrics, columns=['precision', 'recall', 'F1', 'support']).rename(index=id2label))

# avg_n_run_metrics = pd.DataFrame(np.mean(np.asarray(n_run_metrics), axis=0), columns=['precision','recall','F1','support'], index=string_labels)
# print('\nAverge Metrics')
# print(avg_n_run_metrics)

Run 1


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Epoch 1/50 — Train Loss: 2.3344 — Val Loss: 1.5900
Epoch 2/50 — Train Loss: 2.1719 — Val Loss: 1.6297
Epoch 3/50 — Train Loss: 2.1194 — Val Loss: 1.6889
Epoch 4/50 — Train Loss: 2.1051 — Val Loss: 1.7280
Early stopping triggered.

Epoch 1/50 — Train Loss: 1.2174 — Val Loss: 2.9743
Epoch 2/50 — Train Loss: 0.9010 — Val Loss: 2.1205
Epoch 3/50 — Train Loss: 0.9405 — Val Loss: 3.3795
Epoch 4/50 — Train Loss: 1.0331 — Val Loss: 3.9599
Epoch 5/50 — Train Loss: 1.7221 — Val Loss: 3.5315
Early stopping triggered.

Epoch 1/50 — Train Loss: 0.6902 — Val Loss: 2.4986
Epoch 2/50 — Train Loss: 0.5596 — Val Loss: 2.8324
Epoch 3/50 — Train Loss: 0.4625 — Val Loss: 2.9708
Epoch 4/50 — Train Loss: 0.4014 — Val Loss: 2.5069
Early stopping triggered.

Epoch 1/50 — Train Loss: 0.9905 — Val Loss: 3.5944
Epoch 2/50 — Train Loss: 0.6927 — Val Loss: 3.8028
Epoch 3/50 — Train Loss: 0.5617 — Val Loss: 4.6078
Epoch 4/50 — Train Loss: 0.8148 — Val Loss: 3.6035
Early stopping triggered.

Epoch 1/50 — Train Loss: 

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [ ]:
from google.colab import runtime
runtime.unassign()